In [10]:
!pip install transformers
!pip install torch
!pip install accelerate -U

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import pandas as pd
import torch
from transformers import BartTokenizer, BartForSequenceClassification, Trainer, TrainingArguments, Dataloader

ImportError: cannot import name 'Dataloader' from 'transformers' (/opt/conda/lib/python3.10/site-packages/transformers/__init__.py)

In [3]:
main_df = pd.read_csv("/kaggle/input/cyberbullying-classification/cyberbullying_tweets.csv")

In [4]:
df = main_df[['tweet_text','cyberbullying_type']]
print(df.shape)
df.head(5)

(47692, 2)


,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [5]:
df["cyberbullying_type"].value_counts()

cyberbullying_type
religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: count, dtype: int64

In [6]:
train_df = df.sample(frac=0.8, random_state=42)
val_df = df.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

train_encodings = tokenizer(train_df['tweet_text'].tolist(), truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_df['tweet_text'].tolist(), truncation=True, padding=True, max_length=256)


In [7]:
train_labels = train_df['cyberbullying_type'].map({'not_cyberbullying': 0, 'religion': 1, 'age': 2, 'gender': 3, 'ethnicity': 4, 'other_cyberbullying': 5}).tolist()
val_labels = val_df['cyberbullying_type'].map({'not_cyberbullying': 0, 'religion': 1, 'age': 2, 'gender': 3, 'ethnicity': 4, 'other_cyberbullying': 5}).tolist()


In [8]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [9]:
model = BartForSequenceClassification.from_pretrained('facebook/bart-large', num_labels=6)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias', 'classification_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
print(torch.cuda.is_available())

True


In [11]:
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    do_train=True,
    do_eval=True,
    use_cpu=False,
    load_best_model_at_end=True,
    warmup_steps=200,
    weight_decay=0.01,
    logging_first_step=False,
    output_dir='/path/to/your/save/dir'

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.251900,0.408921
2,0.202100,0.345448
3,0.177300,0.390602


TrainOutput(global_step=14310, training_loss=0.3872643305279007, metrics={'train_runtime': 15495.2687, 'train_samples_per_second': 7.387, 'train_steps_per_second': 0.924, 'total_flos': 6.219839698371994e+16, 'train_loss': 0.3872643305279007, 'epoch': 3.0})

In [12]:
torch.save(model.state_dict(), "working_path_6.pth")

In [13]:
import tkinter as tk
from tkinter import messagebox

import torch
from transformers import BartTokenizer, BartForSequenceClassification

LABELS = {
    0: "non-cyberbullying",
    1: "religion",
    2: "age",
    3: "gender",
    4: "ethnicity",
    5: "general-cyberbullying"
}

# Load the model and tokenizer 
model_path = "working_path_6.pth"
model = BartForSequenceClassification.from_pretrained('facebook/bart-large', num_labels=6)
model.load_state_dict(torch.load(model_path))
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")


def predict_cyberbullying_category(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Determine the device to use
    
    model.to(device)  # Move model to the determined device

    # Tokenize input text
    inputs = tokenizer(
        text, 
        max_length=128, 
        truncation=True, 
        padding='max_length', 
        return_attention_mask=True, 
        return_tensors='pt'
    )
    
    # Move inputs to the determined device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Ensure the model is in evaluation mode
    model.eval()

    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predicted_class = torch.argmax(probabilities, dim=-1).item()
    
    return LABELS[predicted_class]

def on_submit():
    text = entry.get("1.0", tk.END).strip()  
    if not text:  
        messagebox.showwarning("No Input", "Please enter text to analyze.")  
        return 
    prediction = predict_cyberbullying_category(text)
    messagebox.showinfo("Prediction", f"The text is classified as: {prediction}")

# Create the main window
root = tk.Tk()
root.title("Cyberbullying Prediction")

# Create a Label
label = tk.Label(root, text="Enter text:")
label.pack(pady=20)

# Create an Entry widget to take user input
entry = tk.Text(root, height=10, width=50)
entry.pack(pady=20)

# Create a Button to submit input
submit_button = tk.Button(root, text="Predict", command=on_submit)
submit_button.pack(pady=20)

root.mainloop()


Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias', 'classification_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TclError: no display name and no $DISPLAY environment variable

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset, DataLoader

model_path = "/kaggle/input/workingpath/working_path_6.pth"
model = BartForSequenceClassification.from_pretrained('facebook/bart-large', num_labels=6)
model.load_state_dict(torch.load(model_path))

def evaluate(model, dataloader, device):
    print("Starting the evaluation...")  # DEBUGGING PRINT
    model.eval()
    total_predictions, total_true = [], []
    with torch.no_grad():
        for batch in dataloader:
            inputs = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(inputs)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1).tolist()

            total_predictions.extend(predictions)
            total_true.extend(labels.tolist())

    accuracy = accuracy_score(total_true, total_predictions)
    precision = precision_score(total_true, total_predictions, average='weighted')
    recall = recall_score(total_true, total_predictions, average='weighted')
    f1 = f1_score(total_true, total_predictions, average='weighted')

    return accuracy * 100, precision, recall, f1

# 2. Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 3. Create the DataLoader for your validation dataset
validation_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 4. Call the evaluate function
accuracy, precision, recall, f1 = evaluate(model, validation_dataloader, device)

# 5. Print the returned metrics
print(f"Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting the evaluation...
Accuracy: 51.89%
Precision: 0.68
Recall: 0.52
F1 Score: 0.46
